In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

### 1 Daten einlesen 

#### 1.1 Verkehrsdaten einlesen

In [6]:
traffic_data = pd.read_csv(r'C:\Users\Admin\Desktop\EGM_2025\data\Cleaned\verkehrszaehlungenUndMessstandorte.csv', delimiter =',', encoding='utf-8')

# Überprüfen der ersten Zeilen
print("\nÜberblick der Verkehrsdaten:")
print(traffic_data.head())
print(f"Anzahl der Datensätze: {len(traffic_data)}")
print(f"Spalten: {traffic_data.columns.tolist()}")


Überblick der Verkehrsdaten:
         date  fuss_in  fuss_out          location_name  \
0  2020-01-01    810.0     758.0  Altstetterstrasse Ost   
1  2020-01-02    982.0     930.0  Altstetterstrasse Ost   
2  2020-01-03   1469.0    1357.0  Altstetterstrasse Ost   
3  2020-01-04   1051.0    1205.0  Altstetterstrasse Ost   
4  2020-01-05    976.0     983.0  Altstetterstrasse Ost   

                            geometry     direction_in direction_out  \
0  POINT (8.4875018197 47.390989781)  Bhf. Altstetten   Lindenplatz   
1  POINT (8.4875018197 47.390989781)  Bhf. Altstetten   Lindenplatz   
2  POINT (8.4875018197 47.390989781)  Bhf. Altstetten   Lindenplatz   
3  POINT (8.4875018197 47.390989781)  Bhf. Altstetten   Lindenplatz   
4  POINT (8.4875018197 47.390989781)  Bhf. Altstetten   Lindenplatz   

                          geometryMessstandort  
0  POINT (8.487499519616366 47.39099069038348)  
1  POINT (8.487499519616366 47.39099069038348)  
2  POINT (8.487499519616366 47.3909906903

#### 1.2 Bevölkerungsdaten einlesen

In [2]:
population_data = pd.read_csv(r'C:\Users\Admin\Desktop\EGM_2025\data\Cleaned\GeschlechtAlterQuartiere_65plus.csv', delimiter=',', encoding='utf-8')

# Überprüfen der ersten Zeilen
print("\nÜberblick der Bevölkerungsdaten:")
print(population_data.head())
print(f"Anzahl der Datensätze: {len(population_data)}")
print(f"Spalten: {population_data.columns.tolist()}")


Überblick der Bevölkerungsdaten:
   Year AgeGroup  Kreis Sex  Origin  PopulationCount
0  1993    65-69      1   M       1               93
1  1993    65-69      1   M       2                9
2  1993    65-69      1   W       1              128
3  1993    65-69      1   W       2               13
4  1993    65-69      2   M       1              663
Anzahl der Datensätze: 10379
Spalten: ['Year', 'AgeGroup', 'Kreis', 'Sex', 'Origin', 'PopulationCount']


### 2. Datenaufbereitung

#### 2.1 Verkehrsdaten aufbereiten

In [8]:
# Extrahieren der Koordinaten aus dem POINT-String
def extract_coordinates(point_str):
    try:
        # Entfernen von 'POINT (' und ')'
        coords = point_str.replace('POINT (', '').replace(')', '')
        lon, lat = coords.split(' ')
        return float(lon), float(lat)
    except:
        return None, None

# Koordinaten extrahieren
traffic_data['lon'], traffic_data['lat'] = zip(*traffic_data['geometry'].apply(extract_coordinates))

# Standorte aggregieren (Durchschnittliche Fussgängerzahlen pro Standort)
location_stats = traffic_data.groupby('location_name').agg({
    'fuss_in': 'mean',
    'fuss_out': 'mean',
    'lon': 'first',
    'lat': 'first'
}).reset_index()

# Gesamtfussgängerzahl berechnen
location_stats['total_pedestrians'] = location_stats['fuss_in'] + location_stats['fuss_out']

print("\nAggregierte Standortstatistiken:")
print(location_stats.head())


Aggregierte Standortstatistiken:
            location_name      fuss_in     fuss_out       lon        lat  \
0   Altstetterstrasse Ost  1566.665254  1140.144068  8.487502  47.390990   
1  Altstetterstrasse West   419.139535   490.790698  8.487287  47.391009   
2               Arboretum  2500.362745  1450.927451  8.537022  47.362581   
3          Cassiopeiasteg   737.462382   587.590387  8.537241  47.342856   
4           Chorherrenweg   103.969828    93.838362  8.498451  47.401319   

   total_pedestrians  
0        2706.809322  
1         909.930233  
2        3951.290196  
3        1325.052769  
4         197.808190  


#### 2.2 Bevölkerungsdaten aufbereiten

In [3]:
# Filtern für die letzten Jahre (2020-2024)
recent_years = list(range(2020, 2025))
recent_population = population_data[population_data['Year'].isin(recent_years)]

# Nach Kreis und Jahr gruppieren (nur 65+ Altersgruppen)
senior_pop = recent_population[recent_population['AgeGroup'].str.contains('65')]
kreis_year_pop = senior_pop.groupby(['Kreis', 'Year'])['PopulationCount'].sum().reset_index()

# Pivot-Tabelle erstellen, um die Entwicklung zu sehen
pop_pivot = kreis_year_pop.pivot(index='Kreis', columns='Year', values='PopulationCount')
pop_pivot['growth_rate'] = ((pop_pivot[2024] - pop_pivot[2020]) / pop_pivot[2020] * 100).round(2)

# Letzte Jahr (2024) als Hauptreferenz
latest_pop = kreis_year_pop[kreis_year_pop['Year'] == 2024].copy()
latest_pop.rename(columns={'PopulationCount': 'Population_65plus'}, inplace=True)

print("\nSeniorenpopulation nach Kreis (2024):")
print(latest_pop.sort_values('Population_65plus', ascending=False).head())


Seniorenpopulation nach Kreis (2024):
    Kreis  Year  Population_65plus
54     11  2024               2682
44      9  2024               2172
34      7  2024               1751
14      3  2024               1648
49     10  2024               1632


In [4]:
print(pop_pivot)

Year   2020  2021  2022  2023  2024  growth_rate
Kreis                                           
1       284   291   274   246   258        -9.15
2      1193  1220  1280  1288  1299         8.89
3      1580  1596  1594  1626  1648         4.30
4       844   869   873   856   886         4.98
5       408   431   460   504   519        27.21
6      1210  1188  1176  1232  1242         2.64
7      1787  1777  1743  1703  1751        -2.01
8       717   729   747   753   737         2.79
9      2072  2099  2133  2181  2172         4.83
10     1556  1589  1619  1630  1632         4.88
11     2625  2638  2592  2625  2682         2.17
12     1218  1208  1194  1212  1204        -1.15


### 3. DBSCAN-Analyse für Verkehrshotspots

In [10]:
# 3.1 Koordinaten für DBSCAN vorbereiten
X = location_stats[['lon', 'lat']].values

# 3.2 DBSCAN durchführen (eps = 0.003 entspricht ca. 300m)
eps_value = 0.003  # ca. 300m in Grad
min_samples = 2

db = DBSCAN(eps=eps_value, min_samples=min_samples, metric='haversine').fit(X)
labels = db.labels_

# Cluster-Labels zu den Daten hinzufügen
location_stats['cluster'] = labels
location_stats['is_hotspot'] = (labels != -1).astype(int)  # -1 = Noise

# Hotspot-Intensität basierend auf Fussgängeraufkommen
def get_intensity(pedestrians):
    if pedestrians > 3000:
        return "high"
    elif pedestrians > 1500:
        return "medium"
    else:
        return "low"

location_stats['hotspot_intensity'] = location_stats.apply(
    lambda row: get_intensity(row['total_pedestrians']) if row['is_hotspot'] == 1 else "none", 
    axis=1
)

print("\nAnzahl der identifizierten Verkehrshotspots:", sum(location_stats['is_hotspot']))
print(location_stats[['location_name', 'cluster', 'is_hotspot', 'hotspot_intensity']].head(10))


Anzahl der identifizierten Verkehrshotspots: 13
            location_name  cluster  is_hotspot hotspot_intensity
0   Altstetterstrasse Ost        0           1            medium
1  Altstetterstrasse West        0           1               low
2               Arboretum       -1           0              none
3          Cassiopeiasteg       -1           0              none
4           Chorherrenweg        1           1               low
5                 Dammweg        2           1            medium
6              Fischerweg        3           1               low
7             Hardeggsteg       -1           0              none
8              Katzenbach       -1           0              none
9        Kloster-Fahr-Weg        3           1               low


### 4. Demographische Hotspots identifizieren

In [11]:
# 4.1 Durchschnitt und Median berechnen
avg_population = latest_pop['Population_65plus'].mean()
median_population = latest_pop['Population_65plus'].median()

# 4.2 Demographische Hotspots definieren
latest_pop['is_demographic_hotspot'] = (latest_pop['Population_65plus'] > avg_population).astype(int)

# 4.3 Intensität berechnen
def get_demographic_intensity(population, avg):
    if population > avg * 1.5:
        return "high"
    elif population > avg:
        return "medium"
    else:
        return "low"

latest_pop['demographic_hotspot_intensity'] = latest_pop.apply(
    lambda row: get_demographic_intensity(row['Population_65plus'], avg_population),
    axis=1
)

print("\nDemographische Hotspots (Durchschnitt: {:.0f}):".format(avg_population))
print(latest_pop.sort_values('Population_65plus', ascending=False)[
    ['Kreis', 'Population_65plus', 'is_demographic_hotspot', 'demographic_hotspot_intensity']
])


Demographische Hotspots (Durchschnitt: 1336):
    Kreis  Population_65plus  is_demographic_hotspot  \
54     11               2682                       1   
44      9               2172                       1   
34      7               1751                       1   
14      3               1648                       1   
49     10               1632                       1   
9       2               1299                       0   
29      6               1242                       0   
59     12               1204                       0   
19      4                886                       0   
39      8                737                       0   
24      5                519                       0   
4       1                258                       0   

   demographic_hotspot_intensity  
54                          high  
44                          high  
34                        medium  
14                        medium  
49                        medium  
9             

### 5. CSV-Dateien speichern

In [12]:
# 5.1 Verkehrshotspots CSV
traffic_hotspots_df = location_stats[[
    'location_name', 'lon', 'lat', 'total_pedestrians', 
    'cluster', 'is_hotspot', 'hotspot_intensity'
]]

traffic_hotspots_df.to_csv('traffic_hotspots.csv', index=False)

# 5.2 Demographische Hotspots CSV
demographic_hotspots_df = latest_pop[[
    'Kreis', 'Population_65plus', 'is_demographic_hotspot', 'demographic_hotspot_intensity'
]]

demographic_hotspots_df.to_csv('demographic_hotspots.csv', index=False)

print("CSV-Dateien wurden gespeichert.")



CSV-Dateien wurden gespeichert.
